In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import torch
from torch import optim, nn
from math import exp

import numpy as np

from tqdm import tqdm
from random import randint

from load_dataset import load_dataset, plot_image
from auto_encoder2 import PAutoE

device = "cuda"

pimages = load_dataset().to(device)

In [2]:
T = 1000
beta = torch.linspace(10e-4, 0.02, T, device=device)

alpha = 1 - beta

alpha_ = torch.zeros(T, device=device)

for i in range(T):
    alpha_[i] = torch.prod(alpha[0:i+1])

alpha_[-1]

tensor(2.5652e-05, device='cuda:0')

In [ ]:
pimages[0].shape

In [ ]:
a = torch.randperm(3)
a.tolist()

In [ ]:
t = -1

a = torch.randperm(3)
x0 = pimages[0, 0, a]

z = torch.randn(x0.shape).to(device)
xt = x0 * torch.sqrt(alpha_[t]) + z * torch.sqrt(1-alpha_[t])

plot_image(x0)
plot_image(xt)

In [ ]:
for t in range(597,T):
    
    model = PAutoE(3, 3).to(device)

    loss_func = nn.MSELoss()    
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
        
    pbar = tqdm(range(15001), miniters=15)
    def closure():
        
        optimizer.zero_grad()

        source = randint(0, 2)
        batch = torch.randperm(385)[:8]
        color = torch.randperm(3)
        
        x0 = pimages[source, batch][color]
        e = torch.randn(x0.shape, device=device)
        
        x_nois = torch.sqrt(alpha_[t]) * x0 + torch.sqrt(1-alpha_[t]) * e
        e_pred = model(x_nois)

        # Calculate the loss between the result and the noise
        loss = loss_func(e, e_pred)
        loss.backward()
        
        pbar.set_description(f"t = {t}, %.8f" % loss, refresh=False)
        
        return loss
            
    model.train()
    for j in pbar:
        optimizer.step(closure)

    script = torch.jit.script(model)
    script.save(f"./M/model_{t}.pt")

t = 978, 0.00194463:  21%|███▋             | 3210/15001 [00:38<02:21, 83.17it/s]

In [ ]:
from UNet import UNet

In [ ]:
model = UNet(3,3,8).to(device)

In [ ]:
def noise_images(x, t):
    "Add noise to images at instant t"
    a = torch.sqrt(alpha_[t])[:, None, None, None]
    b = torch.sqrt(1 - alpha_[t])[:, None, None, None]
    Ɛ = torch.randn_like(x)
    return a * x + b * Ɛ, Ɛ

batch_size = 8

t = torch.randint(low=1, high=13, size=(batch_size,), device=device)

source = randint(0, 2)
batch = torch.randperm(385)[:batch_size]
x0 = pimages[source][batch]

x_t, noise = noise_images(x0, t)

model(x_t, t)

In [ ]:
torch.randint(low=1, high=13, size=(3,))

In [ ]:
t

In [ ]:
3 //2